# Basic Message - Alice

**Should be run alongside [Bob](http://localhost:8889/lab/tree/3%20Messages/Part%201%20-%20Basic%20Message.ipynb)**

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

In [3]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## Setup a listener for the basicmessages topic

This is emitted using PyPubSub when the controller receives a basicmessages webhook from the agent. This happens everytime the agent receives a basicmessage.

In [4]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


In [5]:
def messages_handler2(payload):
    connection_id = payload["connection_id"]
    print("Second Handler", payload, connection_id)


message_listener2 = {
    "handler": messages_handler2,
    "topic": "basicmessages"
}

agent_controller.add_listener(message_listener2)

Subscribing too: basicmessages


## Verify an Active Connection Exists

This should have been established through the setup docker container using the create_connection.py script.

In [6]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'rfc23_state': 'completed', 'their_label': 'Bob', 'my_did': '78JbknZS4Un2nRsSXthrrX', 'created_at': '2023-01-01 15:37:41.837136Z', 'routing_state': 'none', 'connection_id': 'c3d00305-0806-4846-954d-934ac0c36533', 'invitation_mode': 'once', 'accept': 'manual', 'updated_at': '2023-01-01 15:38:46.703546Z', 'invitation_key': 'C9VVc5HS8ASbZGJ4RRH38huh7jxzV1Lxc6dv9MMhYRm6', 'state': 'active', 'their_role': 'invitee', 'their_did': '6UJzVcKRUTNd2Z3fkgSwqc'}]
Connection : {'rfc23_state': 'completed', 'their_label': 'Bob', 'my_did': '78JbknZS4Un2nRsSXthrrX', 'created_at': '2023-01-01 15:37:41.837136Z', 'routing_state': 'none', 'connection_id': 'c3d00305-0806-4846-954d-934ac0c36533', 'invitation_mode': 'once', 'accept': 'manual', 'updated_at': '2023-01-01 15:38:46.703546Z', 'invitation_key': 'C9VVc5HS8ASbZGJ4RRH38huh7jxzV1Lxc6dv9MMhYRm6', 'state': 'active', 'their_role': 'invitee', 'their_did': '6UJzVcKRUTNd2Z3fkgSwqc'}
Active Connection ID :  c3d00305-0806-4846-954d-934ac0c36533


In [7]:
print(connection["connection_id"])
id = connection["connection_id"]

c3d00305-0806-4846-954d-934ac0c36533


In [8]:
await agent_controller.messaging.trust_ping(id, "hello, world!")

{'thread_id': '9666d644-93f9-49af-b85f-524607d33edc'}

Handle message {'connection_id': 'c3d00305-0806-4846-954d-934ac0c36533', 'message_id': '21e3f400-57ee-47c6-9826-9ee4781543e6', 'content': 'Hello from Bob', 'state': 'received'} c3d00305-0806-4846-954d-934ac0c36533
Second Handler {'connection_id': 'c3d00305-0806-4846-954d-934ac0c36533', 'message_id': '21e3f400-57ee-47c6-9826-9ee4781543e6', 'content': 'Hello from Bob', 'state': 'received'} c3d00305-0806-4846-954d-934ac0c36533


## Send a Basic Message over DIDComm to Bob

See [aries-rfc](https://github.com/hyperledger/aries-rfcs/tree/master/features/0095-basic-message)

You can send as messages as you want, if you are running the [basic-message tutorial on Bob's notebook](http://localhost:8889/lab/tree/3%20Messages/Part%201%20-%20Basic%20Message.ipynb) these will be received and printed by the message handler. You may have to run a code block to see the output.

In [10]:
basic_message = "hello Bob"
response = await agent_controller.messaging.send_message(connection_id, basic_message)
print("BASIC MESSAGE - Alice -> Bob")
print(response)

BASIC MESSAGE - Alice -> Bob
{}
Handle message {'connection_id': 'c3d00305-0806-4846-954d-934ac0c36533', 'message_id': '1ec94969-172c-4c84-93d3-7e76de1d2654', 'content': 'This is a response from Bob', 'state': 'received'} c3d00305-0806-4846-954d-934ac0c36533
Second Handler {'connection_id': 'c3d00305-0806-4846-954d-934ac0c36533', 'message_id': '1ec94969-172c-4c84-93d3-7e76de1d2654', 'content': 'This is a response from Bob', 'state': 'received'} c3d00305-0806-4846-954d-934ac0c36533


## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [11]:
response = await agent_controller.terminate()
print(response)

None
